In [1]:
import sys
sys.path.append("/home/srelins/docs/cy_fdm_builder/src")
from testing_helpers import *
from FDM_builder import *

## TO DO:

* PRIORITY: TEST observation_period table!!
- in particular, work our what happens if you re-run .build() on a dataset


* split table and dataset objects into two sparate source files
* start process with dataset - add guidance messages for dataset creation
i.e. dataset empty, build FDMTables -- or -- dataset contain XXXXX etc...
* run dataset build over whole dataset by default - add options to specify table_ids/FDMTable objects
* will need to add "verbose" functionality to FDMTables or the console print statements will be fucking epic
* add .drop() method to table


## Testing:

In [ ]:
build_test_environment()

In [3]:
table_1 = FDMTable(
    source_table_full_id=f"{PROJECT}.CY_TESTS_SRC.src_table_1",
    dataset_id="CY_TESTS_FDM"
)

	 ##### Initialising FDM Table - src_table_1 #####
________________________________________________________________________________

	* src_table_1 already exists in CY_TESTS_FDM.

	NOTE: Working from the existing version of src_table_1
	in CY_TESTS_FDM. If you wish to begin from scratch with a
	fresh copy, drop the existing table in CY_TESTS_FDM and run
	.build() again.



In [4]:
table_1.build()

	 ##### BUILDING FDM TABLE COMPONENTS FOR src_table_1 #####
________________________________________________________________________________

1. Checking for identifiers and correcting syntax:

	* person_id found - syntax correct

2. Adding person_id column:

	* src_table_1 already contains person_id column

3. Adding event_start_date:

	* event_start_date column already present.

	NOTE: If you wish to rebuild the event start date column, drop event_start_date
	and run .build(...) again.

4. Adding event_end_date:

	* event_end_date column already present.

	NOTE: If you wish to rebuild the event end date column, drop event_end_date
	and run .build(...) again.

________________________________________________________________________________

	 ##### BUILD PROCESS FOR src_table_1 COMPLETE! #####



In [5]:
table_1.get_column_names()

['event_end_date', 'event_start_date', 'person_id', 'date']

In [6]:
table_1.build(event_start_date_cols="date")

	 ##### BUILDING FDM TABLE COMPONENTS FOR src_table_1 #####
________________________________________________________________________________

1. Checking for identifiers and correcting syntax:

	* person_id found - syntax correct

2. Adding person_id column:

	* src_table_1 already contains person_id column

3. Adding event_start_date:

	* event_start_date column already present.

	NOTE: If you wish to rebuild the event start date column, drop event_start_date
	and run .build(...) again.

4. Adding event_end_date:

	* event_end_date column already present.

	NOTE: If you wish to rebuild the event end date column, drop event_end_date
	and run .build(...) again.

________________________________________________________________________________

	 ##### BUILD PROCESS FOR src_table_1 COMPLETE! #####



In [7]:
table_1.build(event_end_date_cols="date")

	 ##### BUILDING FDM TABLE COMPONENTS FOR src_table_1 #####
________________________________________________________________________________

1. Checking for identifiers and correcting syntax:

	* person_id found - syntax correct

2. Adding person_id column:

	* src_table_1 already contains person_id column

3. Adding event_start_date:

	* event_start_date column already present.

	NOTE: If you wish to rebuild the event start date column, drop event_start_date
	and run .build(...) again.

4. Adding event_end_date:

	* event_end_date column already present.

	NOTE: If you wish to rebuild the event end date column, drop event_end_date
	and run .build(...) again.

________________________________________________________________________________

	 ##### BUILD PROCESS FOR src_table_1 COMPLETE! #####



In [8]:
table_2 = FDMTable(
    source_table_full_id=f"{PROJECT}.CY_TESTS_SRC.src_table_2",
    dataset_id="CY_TESTS_FDM"
)

	 ##### Initialising FDM Table - src_table_2 #####
________________________________________________________________________________

	* src_table_2 already exists in CY_TESTS_FDM.

	NOTE: Working from the existing version of src_table_2
	in CY_TESTS_FDM. If you wish to begin from scratch with a
	fresh copy, drop the existing table in CY_TESTS_FDM and run
	.build() again.



In [9]:
table_2.build()

	 ##### BUILDING FDM TABLE COMPONENTS FOR src_table_2 #####
________________________________________________________________________________

1. Checking for identifiers and correcting syntax:

	* person_id found - syntax correct
	* digest found - syntax correct

2. Adding person_id column:

	* src_table_2 already contains person_id column

	NOTE: digest also found in src_table_2. If you
	wish to rebuild the person_id column from digest, drop the existing
	person_id column in src_table_2 and run .build() again

3. Adding event_start_date:

	* event_start_date column already present.

	NOTE: If you wish to rebuild the event start date column, drop event_start_date
	and run .build(...) again.

4. Adding event_end_date:

	* event_end_date column already present.

	NOTE: If you wish to rebuild the event end date column, drop event_end_date
	and run .build(...) again.

________________________________________________________________________________

	 ##### BUILD PROCESS FOR src_table_2 COM

In [10]:
table_2.build(event_start_date_cols=["year", "month", "day"])

	 ##### BUILDING FDM TABLE COMPONENTS FOR src_table_2 #####
________________________________________________________________________________

1. Checking for identifiers and correcting syntax:

	* person_id found - syntax correct
	* digest found - syntax correct

2. Adding person_id column:

	* src_table_2 already contains person_id column

	NOTE: digest also found in src_table_2. If you
	wish to rebuild the person_id column from digest, drop the existing
	person_id column in src_table_2 and run .build() again

3. Adding event_start_date:

	* event_start_date column already present.

	NOTE: If you wish to rebuild the event start date column, drop event_start_date
	and run .build(...) again.

4. Adding event_end_date:

	* event_end_date column already present.

	NOTE: If you wish to rebuild the event end date column, drop event_end_date
	and run .build(...) again.

________________________________________________________________________________

	 ##### BUILD PROCESS FOR src_table_2 COM

In [11]:
table_2.build(event_end_date_cols=["year", "month", "day"])

	 ##### BUILDING FDM TABLE COMPONENTS FOR src_table_2 #####
________________________________________________________________________________

1. Checking for identifiers and correcting syntax:

	* person_id found - syntax correct
	* digest found - syntax correct

2. Adding person_id column:

	* src_table_2 already contains person_id column

	NOTE: digest also found in src_table_2. If you
	wish to rebuild the person_id column from digest, drop the existing
	person_id column in src_table_2 and run .build() again

3. Adding event_start_date:

	* event_start_date column already present.

	NOTE: If you wish to rebuild the event start date column, drop event_start_date
	and run .build(...) again.

4. Adding event_end_date:

	* event_end_date column already present.

	NOTE: If you wish to rebuild the event end date column, drop event_end_date
	and run .build(...) again.

________________________________________________________________________________

	 ##### BUILD PROCESS FOR src_table_2 COM

In [12]:
table_3 = FDMTable(
    source_table_full_id=f"{PROJECT}.CY_TESTS_SRC.src_table_3",
    dataset_id="CY_TESTS_FDM"
)

	 ##### Initialising FDM Table - src_table_3 #####
________________________________________________________________________________

	* src_table_3 already exists in CY_TESTS_FDM.

	NOTE: Working from the existing version of src_table_3
	in CY_TESTS_FDM. If you wish to begin from scratch with a
	fresh copy, drop the existing table in CY_TESTS_FDM and run
	.build() again.



In [13]:
table_3.build()

	 ##### BUILDING FDM TABLE COMPONENTS FOR src_table_3 #####
________________________________________________________________________________

1. Checking for identifiers and correcting syntax:

	* person_id found - syntax correct
	* EDRN found - syntax correct

2. Adding person_id column:

	* src_table_3 already contains person_id column

	NOTE: EDRN also found in src_table_3. If you
	wish to rebuild the person_id column from EDRN, drop the existing
	person_id column in src_table_3 and run .build() again

3. Adding event_start_date:

	No event_start_date column found and event_start_date_cols not specified.

	To complete the build process, an event start date is required. Specify the
	`event_start_date_cols` argument of the .build() method as either:

	* If one column contains the full event date:

	  run .build(event_start_date_cols="insert-column-name-here")
	  NOTE: The process assumes a default date format of year-month-day. If the
	  date column is formatted as day-month-year set 

In [14]:
table_3.get_column_names()

['person_id', 'EDRN', 'month', 'year']

In [15]:
table_3.build(event_start_date_cols=["year", "month", "15"])

	 ##### BUILDING FDM TABLE COMPONENTS FOR src_table_3 #####
________________________________________________________________________________

1. Checking for identifiers and correcting syntax:

	* person_id found - syntax correct
	* EDRN found - syntax correct

2. Adding person_id column:

	* src_table_3 already contains person_id column

	NOTE: EDRN also found in src_table_3. If you
	wish to rebuild the person_id column from EDRN, drop the existing
	person_id column in src_table_3 and run .build() again

3. Adding event_start_date:

	* event_start_date column added!

4. Adding event_end_date:

	No event_end_date column found and event_end_date_column not specified.
	event_end_date end date will be omitted. If you wish to add an event_end_date,
	re-run .build() specifying the `event_end_date` argument.

	Note: Table build process will complete, but the observation periods derived
	from a source table with only an event start date may not be accurate, and 
	may result in erroneous obser

In [16]:
table_3.build(event_end_date_cols=["year", "month", "18"])

	 ##### BUILDING FDM TABLE COMPONENTS FOR src_table_3 #####
________________________________________________________________________________

1. Checking for identifiers and correcting syntax:

	* person_id found - syntax correct
	* EDRN found - syntax correct

2. Adding person_id column:

	* src_table_3 already contains person_id column

	NOTE: EDRN also found in src_table_3. If you
	wish to rebuild the person_id column from EDRN, drop the existing
	person_id column in src_table_3 and run .build() again

3. Adding event_start_date:

	* event_start_date column already present.

	NOTE: If you wish to rebuild the event start date column, drop event_start_date
	and run .build(...) again.

4. Adding event_end_date:

	* event_end_date column added!

________________________________________________________________________________

	 ##### BUILD PROCESS FOR src_table_3 COMPLETE! #####



In [17]:
table_4 = FDMTable(
    source_table_full_id = f"{PROJECT}.CY_TESTS_SRC.src_table_4",
    dataset_id="CY_TESTS_FDM"
)

	 ##### Initialising FDM Table - src_table_4 #####
________________________________________________________________________________

* Table src_table_4 copied to CY_TESTS_FDM!


In [18]:
table_4.head()

,person_id,digest,date_string
0,48,@N,24-12-2029
1,30,?F,7-8-1989
2,47,@M,24-11-2028
3,31,?G,23-6-1988
4,37,?M,22-12-2005
5,35,?K,14-3-2005
6,43,@I,27-9-2009
7,33,?I,17-10-1989
8,junk_person_id_314,junk_digest_314,16-8-2002
9,41,@G,3-4-2002


In [37]:
table_4.build(event_start_date_cols="date_string", 
              event_end_date_cols="date_string")
table_4.head()

	 ##### BUILDING FDM TABLE COMPONENTS FOR src_table_4 #####
________________________________________________________________________________

1. Checking for identifiers and correcting syntax:

	* person_id found - syntax correct
	* digest found - syntax correct

2. Adding person_id column:

	* src_table_4 already contains person_id column

	NOTE: digest also found in src_table_4. If you
	wish to rebuild the person_id column from digest, drop the existing
	person_id column in src_table_4 and run .build() again

3. Adding event_start_date:

	* event_start_date column already present.

	NOTE: If you wish to rebuild the event start date column, drop event_start_date
	and run .build(...) again.

4. Adding event_end_date:

	* event_end_date column added!

________________________________________________________________________________

	 ##### BUILD PROCESS FOR src_table_4 COMPLETE! #####



,event_end_date,event_start_date,person_id,digest,date_string
0,1989-12-10,1989-12-10,34,?J,12-10-1989
1,2004-05-23,2004-05-23,junk_person_id_315,junk_digest_315,23-5-2004
2,2020-01-17,2020-01-17,49,@O,17-1-2020
3,2002-08-16,2002-08-16,junk_person_id_314,junk_digest_314,16-8-2002
4,2004-10-28,2004-10-28,junk_person_id_511,junk_digest_511,28-10-2004
5,1989-07-08,1989-07-08,30,?F,7-8-1989
6,2029-12-24,2029-12-24,48,@N,24-12-2029
7,2005-03-14,2005-03-14,35,?K,14-3-2005
8,2028-12-21,2028-12-21,45,@K,21-12-2028
9,2004-07-06,2004-07-06,44,@J,7-6-2004


In [20]:
table_4.head()

,event_start_date,person_id,digest,date_string
0,2004-07-06,44,@J,7-6-2004
1,2000-08-22,38,?N,22-8-2000
2,2001-02-25,40,@F,25-2-2001
3,2005-12-22,37,?M,22-12-2005
4,2022-10-18,46,@L,18-10-2022
5,2009-09-27,43,@I,27-9-2009
6,1988-06-23,31,?G,23-6-1988
7,2029-12-24,48,@N,24-12-2029
8,2004-05-23,junk_person_id_315,junk_digest_315,23-5-2004
9,2004-04-27,junk_person_id_703,junk_digest_703,27-4-2004


In [21]:
table_5 = FDMTable(
    source_table_full_id = f"{PROJECT}.CY_TESTS_SRC.src_table_5",
    dataset_id = "CY_TESTS_FDM"
)
table_5.build()

	 ##### Initialising FDM Table - src_table_5 #####
________________________________________________________________________________

* Table src_table_5 copied to CY_TESTS_FDM!
	 ##### BUILDING FDM TABLE COMPONENTS FOR src_table_5 #####
________________________________________________________________________________

1. Checking for identifiers and correcting syntax:

	* person_id found - syntax correct
	* EDRN found - syntax correct

2. Adding person_id column:

	* src_table_5 already contains person_id column

	NOTE: EDRN also found in src_table_5. If you
	wish to rebuild the person_id column from EDRN, drop the existing
	person_id column in src_table_5 and run .build() again

3. Adding event_start_date:

	No event_start_date column found and event_start_date_cols not specified.

	To complete the build process, an event start date is required. Specify the
	`event_start_date_cols` argument of the .build() method as either:

	* If one column contains the full event date:

	  run .build

In [22]:
table_5.head()

,person_id,EDRN,date_string
0,48,JX,July-2000
1,40,JP,February-1983
2,57,KW,April-2026
3,52,KR,May-2001
4,junk_person_id_358,junk_EDRN_358,November-2009
5,55,KU,October-2021
6,45,JU,September-2002
7,53,KS,July-2005
8,47,JW,February-2003
9,42,JR,June-1986


In [23]:
table_5.add_column('SPLIT(date_string, "-")[OFFSET(0)] AS Month')
table_5.head()

,person_id,EDRN,date_string,Month
0,49,JY,May-2006,May
1,52,KR,May-2001,May
2,48,JX,July-2000,July
3,54,KT,July-2009,July
4,53,KS,July-2005,July
5,51,KQ,July-2003,July
6,44,JT,July-1982,July
7,42,JR,June-1986,June
8,57,KW,April-2026,April
9,junk_person_id_567,junk_EDRN_567,August-2005,August


In [24]:
table_5.add_column('SPLIT(date_string, "-")[OFFSET(1)] AS Year')
table_5.head()

,person_id,EDRN,date_string,Month,Year
0,49,JY,May-2006,May,2006
1,52,KR,May-2001,May,2001
2,53,KS,July-2005,July,2005
3,48,JX,July-2000,July,2000
4,44,JT,July-1982,July,1982
5,51,KQ,July-2003,July,2003
6,54,KT,July-2009,July,2009
7,42,JR,June-1986,June,1986
8,57,KW,April-2026,April,2026
9,junk_person_id_567,junk_EDRN_567,August-2005,August,2005


In [25]:
table_5.drop_column("date_string")

AttributeError: 'FDMTable' object has no attribute 'head9'

In [26]:
table_5.head()

,person_id,EDRN,Month,Year
0,49,JY,May,2006
1,52,KR,May,2001
2,53,KS,July,2005
3,48,JX,July,2000
4,44,JT,July,1982
5,51,KQ,July,2003
6,54,KT,July,2009
7,42,JR,June,1986
8,57,KW,April,2026
9,junk_person_id_567,junk_EDRN_567,August,2005


In [32]:
table_5.build(event_start_date_cols = ["15", "Month", "Year"], event_start_date_format="DMY", 
              event_end_date_cols = ["15", "Month", "Year"], event_end_date_format="DMY")

	 ##### BUILDING FDM TABLE COMPONENTS FOR src_table_5 #####
________________________________________________________________________________

1. Checking for identifiers and correcting syntax:

	* person_id found - syntax correct
	* EDRN found - syntax correct

2. Adding person_id column:

	* src_table_5 already contains person_id column

	NOTE: EDRN also found in src_table_5. If you
	wish to rebuild the person_id column from EDRN, drop the existing
	person_id column in src_table_5 and run .build() again

3. Adding event_start_date:

	* event_start_date column already present.

	NOTE: If you wish to rebuild the event start date column, drop event_start_date
	and run .build(...) again.

4. Adding event_end_date:

	* event_end_date column added!

________________________________________________________________________________

	 ##### BUILD PROCESS FOR src_table_5 COMPLETE! #####



In [35]:
dataset = FDMDataset(dataset_id="CY_TESTS_FDM", fdm_tables=[table_1, table_2, table_3, table_4, table_5])

In [38]:
dataset.build()

		 ##### BUILDING FDM DATASET CY_TESTS_FDM #####
________________________________________________________________________________

1. Checking source input tables:

	* src_table_1 - OK
	* src_table_2 - OK
	* src_table_3 - OK
	* src_table_4 - OK
	* src_table_5 - OK

2. Building person table:

	* Joining data from master person table
	* Person table built!

3. Building table of individuals with no person_id

	* individuals_missing_person_id created with 10 entries

4. Building person_ids missing from master table

	* person_ids_missing_from_master created with 15 entries
5. Building initial observation_period table

	* observation_period table built
6. Removing entries outside observation period

	* entries outside observation period removed from src_table_1
	  and stored in src_table_1_outside_obs
	* entries outside observation period removed from src_table_2
	  and stored in src_table_2_outside_obs
	* entries outside observation period removed from src_table_3
	  and stored in src_tabl

In [ ]:
sql = f"""
    WITH possible_dates AS (
        WITH all_src_dates AS (
            SELECT person_id, event_start_date, event_end_date 
            FROM {table_1.full_table_id}
            WHERE person_id IS NOT NULL

            UNION ALL

            SELECT person_id, event_start_date, event_end_date 
            FROM {table_2.full_table_id}
            WHERE person_id IS NOT NULL
            
            UNION ALL

            SELECT person_id, event_start_date, event_end_date 
            FROM {table_3.full_table_id}
            WHERE person_id IS NOT NULL
        )
        SELECT person_id, 
            MIN(event_start_date) AS possible_start_date,
            MAX(event_end_date) AS possible_end_date 
        FROM all_src_dates
        GROUP BY person_id
            
        UNION ALL
            
        SELECT person_id,
            birth_datetime AS possible_start_date, 
            DATETIME_ADD(IFNULL(death_datetime, DATETIME "9999-01-01 00:00:00"), INTERVAL 42 DAY) as possible_end_date 
        FROM {dataset.person_table_id}
    )
    SELECT person_id, 
        MAX(possible_start_date) AS observation_period_start_date,
        MIN(possible_end_date) AS observation_period_end_date
    FROM possible_dates
    GROUP BY person_id
"""
df = pd.read_gbq(sql, project_id=f"{PROJECT}")

In [ ]:
df[df.observation_period_start_date > df.observation_period_end_date]

In [ ]:
sql = f"""
    SELECT *
    FROM {PROJECT}.CY_TESTS_SRC.src_table_1
    WHERE date < DATETIME "1990-12-12 00:00:00" 
    OR date > DATETIME "2010-12-12 00:00:00"
"""
df = pd.read_gbq(sql, project_id=PROJECT)
df